# Calculation of volume constraints for Si-composite anodes

**Authors:**
- Carl Erik L. Foss, IFE
- Jan Petter Maehlen, IFE

**Last update:** 2021.04.23

**Acknowledgements:** Elkem and Research Council of Norway

## Imports and setup

In [ ]:
# from IPython.display import display, Math, Latex
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Information

(or maybe only a link to information)

Fig 1. Expansion

<img src="SiliconExpansion_fig_001.png">

Fig 2. Initial vs. expanded

<img src="SiliconExpanded_fig_001.png">

## DEV: Objective of notebook

1. Calculate porosity based on measured mass, thickness and either area or diamter of electrode.
2. Calculate change in porosity based on lithiation level
3. Should be possible to generate a xy plot of porosity vs lithiation level for any given Silicon/Graphite/Other ratio
4. Should be possible to generate a plot showing volumetric and gravimetric capacity as a function of Silicon/Graphite/Other ratio and lithiation level

In the end, the functions can be moved out of the notebook into a python module (for example a .py file in the same directory)

### Next (other notebook?)
1. Interactive notebook
2. Calculate energy density of "real" cell given material prms for anode and cathode



## DEV: Notes and trix

1. Since github is deprecating the option of using a userename/password authentication, you might have to use ssh

    a. create ssh key and set it up
    
    b. make sure you set origin to the git@github.com:jepegit/coffeshop.git (not the html url that used to be standard)
    
    e.g. git remote set-url origin git@github.com:jepegit/coffeshop.git

## Input

Used for development

In [ ]:
# Needed input prms (constants)

# expansion coeffs
expansion_coeff_si       = 2.8 
expansion_coeff_graphite = 0.1
expansion_coeff_dead     = 0.0

# densities
rho_si = 2.329           # g/ccm
rho_graphite = 2.1       # g/ccm
rho_dead = 1.8           # g/ccm

# max capacities
maxcap_si = 3579.0       # mAh/g
maxcap_graphite = 372.0  # mAh/g
maxcap_dead = 0.0        # mAh/g


In [ ]:
# Needed input prms (variables)

# composition
mass_fraction_silicon  = 0.6
mass_fraction_graphite = 0.3
mass_fraction_dead     = 0.1

# electrode / cell
radius = 0.5*1.5

eps = 1.0  # allowed expansion coef for electrode (1.0 means no room available)

# cut-off capacities
max_cutoff_cap_si = 3579.0       # mAh/g
max_cutoff_cap_graphite = 372.0  # mAh/g
max_cutoff_cap_dead = 0.0        # mAh/g

# porosity range
porosity = 0.5 # initial porosity
porosity_limit = 0.0 # final limit for porosity

## Utility functions

In [ ]:
def density_composite(densities, mass_fractions, porosity = 0.0, debug=False):
    """Calculate density of composite.
    
    Parameters
    ----------
    densities : list of floats
        The densities of the composite components (in g/ccm).
    mass_fractions : list of floats
        The mass fraction of each composite component (in g).
    porosity : float 
        Porosity (from 0.0 to 1.0, where 0.0 means no porosity).
    debug : bool
        Print messages to stdout if True.
        
    Returns
    -------
    Float
        The calculated density
    """
    
    
    total_mass = np.sum(mass_fractions)
    
    total_volume = 0.0
    for d,m in zip(densities, mass_fractions):
        if debug:
            print(f"density: {d}\nfraction: {m}")
        total_volume += m/d
    try:
        total_volume = total_volume/(1-porosity)
    except ZeroDivisionError:
        print("ERROR! You have a porosity of 1.0 (meaning your electrode contains nothing)")
        return None
    
    density = total_mass/total_volume
    if debug:
        print(f"Density: {density}")
    return density

In [ ]:
def density_outer(area, thickness, mass):
    """Calculate the outer density of the electrode.
    
    Parameters
    ----------
    area : float
        The area of the electrode (in cm2).
    thickness : float
        The thickness of the electrode (in cm).
    mass : float 
        mass of the electrode (in g)
        
    Returns
    -------
    Float
        The calculated density
    """
    
    volume = area * thickness
    density = mass/volume
    return density

In [ ]:
def porosity_theoretical(inner_density, outer_density):
    return 1 - (outer_density / inner_density)


# 1. Calculate porosity based on measured mass, thickness and either area or diamter of electrode.

In [ ]:
# Some vars
densities = [rho_si, rho_graphite, rho_dead]
mass_fractions = [mass_fraction_silicon, mass_fraction_graphite, mass_fraction_dead]

mass = 2.23/1000  # g
thickness = 12/10000 # cm
area = 0.25 * np.pi * 1.5**2 # cm2

# test


d1 = density_composite(densities, mass_fractions)
print(f"inner density (composite): {d1}")
d2 = density_outer(area, thickness, mass)
print(f"outer density (outer volume and mass): {d2}")

porosity = porosity_theoretical(d1, d2)

print(f"porosity: {porosity}")


# 2. Calculate change in porosity based on lithiation level

## Notes

- Provide density as a function of lithiation level
- Assume lithiation level is in the range 0.0 to 1.0
- Either: assume linear expansion or provide a function describing the relation

## 2.1 Change in density

In [ ]:
def expansion(lithiation_degree, initial_density, expansion_coeff):
    new_density = 1 / ((1 / initial_density) + expansion_coeff * lithiation_degree)
    return new_density


In [ ]:
lithiation_degree = 1.0
initial_density = 1.0
expansion_coeff = 0.1

expansion(lithiation_degree, initial_density, expansion_coeff)

In [ ]:
initial_density = 2.0
expansion_coeff = 2.8
x = np.linspace(0.0, 1, 100)
y = expansion(x, initial_density, expansion_coeff)

In [ ]:
plt.plot(x,y)

## 2.2 Change in porosity as a function of lithiation degree